In [17]:
!pip install -r requirements.txt > None

In [18]:
RANDOM_STATE = 2023
TRAIN_SET = 50

In [19]:
# https://huggingface.co/abletobetable/distilbert-ru-qa

selected_model = "Den4ikAI/rubert_large_squad_2"

In [34]:
import torch
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [21]:
sber_dataset = load_dataset("sberquad")
squad_dataset = load_dataset("yelp_review_full")

In [22]:
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [23]:
print(squad_dataset["train"][5])

{'label': 4, 'text': "Top notch doctor in a top notch practice. Can't say I am surprised when I was referred to him by another doctor who I think is wonderful and because he went to one of the best medical schools in the country. \\nIt is really easy to get an appointment. There is minimal wait to be seen and his bedside manner is great."}


In [24]:
tokenizer = AutoTokenizer.from_pretrained(selected_model)
tokenizer.is_fast

True

In [28]:
def preprocess_validation_examples(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=240)

In [29]:
test_eval_set = squad_dataset['test'].select(range(TRAIN_SET))
train_eval_set = squad_dataset['train'].select(range(TRAIN_SET))

tokenizer = AutoTokenizer.from_pretrained(selected_model)

train_eval_set = train_eval_set.map(
    preprocess_validation_examples,
    batched=True)

small_eval_set = test_eval_set.map(
    preprocess_validation_examples,
    batched=True)

Map: 100%|██████████| 50/50 [00:00<00:00, 694.42 examples/s]


In [30]:
train_eval_set, small_eval_set

(Dataset({
     features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 50
 }),
 Dataset({
     features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 50
 }))

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(
    selected_model, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Den4ikAI/rubert_large_squad_2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
metric = evaluate.load("accuracy")

In [35]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="TEST_TRAIN", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_eval_set,
    eval_dataset=small_eval_set,
    compute_metrics=compute_metrics,
)


trainer.train()























                                              
                                          

                                        


 33%|███▎      | 7/21 [15:20<06:27, 27.69s/it]






{'eval_loss': 1.9370362758636475, 'eval_accuracy': 0.08, 'eval_runtime': 75.2141, 'eval_samples_per_second': 0.665, 'eval_steps_per_second': 0.093, 'epoch': 1.0}


#### Используя набор данных Sberquad дообучить выбранную модель, оценить качество до и после дообучения